In [1]:
import numpy as np 
import pandas as pd 

In [32]:
data=pd.read_csv("/kaggle/input/grocery-product-prices-for-australian-states/Australia_Grocery_2022Sep.csv")
data.drop(["Postal_code","RunDate","Product_Url","tid","state","index"], axis=1,inplace=True)

In [33]:
data.head(3)

,Category,Sub_category,Product_Group,Product_Name,Package_price,Price_per_unit,package_size,is_estimated,is_special,in_stock,Retail_price,Brand,Sku,unit_price,unit_price_unit,city
0,Meat & seafood,Poultry,Chicken offal,RSPCA Approved Chicken Necks,7.15,$6.50 per 1Kg,approx. 1.1kg,1,0,NaN,NaN,Coles,1491280P,6.5,1Kg,TEMPE
1,Meat & seafood,Poultry,Chicken offal,RSPCA Approved Chicken Livers,5.40,$9.00 per 1Kg,approx. 600g,1,0,NaN,NaN,Coles,1718058P,9.0,1Kg,TEMPE
2,Meat & seafood,Poultry,Chicken offal,RSPCA Approved Chicken Giblets,4.50,$7.50 per 1Kg,approx. 600g,1,0,NaN,NaN,Coles,2565429P,7.5,1Kg,TEMPE


In [34]:
data.shape

(488640, 16)

In [39]:
df=data.dropna()
df=df.reset_index(drop=True)
df

,Category,Sub_category,Product_Group,Product_Name,Package_price,Price_per_unit,package_size,is_estimated,is_special,in_stock,Retail_price,Brand,Sku,unit_price,unit_price_unit,city
0,Meat & seafood,Poultry,Drumsticks & maryland,RSPCA Approved Chicken Drumsticks,7.00,$3.50 per 1Kg,2kg,0,1,False,8.00,Coles,2263260P,3.50,1Kg,TEMPE
1,Meat & seafood,Beef & veal,Beef steaks,Beef Chuck Casserole Steak,17.10,$18.00 per 1Kg,approx. 950g,1,1,False,19.00,Coles,3331172P,18.00,1Kg,TEMPE
2,Meat & seafood,Beef & veal,Slow cook & casserole beef,Beef Chuck Casserole Steak,17.10,$18.00 per 1Kg,approx. 950g,1,1,False,19.00,Coles,3331172P,18.00,1Kg,TEMPE
3,Meat & seafood,Lamb,Lamb steak,Lamb Backstrap,9.92,$40.00 per 1Kg,approx. 248g,1,1,False,11.41,Coles,3411442P,40.00,1Kg,TEMPE
4,Meat & seafood,Pork,Pork chops,Pork Loin Chops,12.00,$15.00 per 1Kg,approx. 800g,1,1,False,14.40,Coles,8600850P,15.00,1Kg,TEMPE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6325,Drinks,Energy drinks,Multipack cans,Sugar Free Energy Drink 250mL,15.50,$7.75 per 1L,8 pack,0,1,False,21.00,Red Bull,3324177P,7.75,1L,CANBERRA CIVIC DOORDASH DUMMY
6326,Drinks,Energy drinks,Multipack cans,Ultra Rosa Multipack Cans 4 x 500mL,8.25,$4.13 per 1L,4 pack,0,1,False,13.75,Monster,3990033P,4.13,1L,CANBERRA CIVIC DOORDASH DUMMY
6327,Drinks,Sports drinks,Sports drink,Fierce Grape Sport Drink,2.25,$3.75 per 1L,600mL,0,1,False,3.80,Gatorade,8729937P,3.75,1L,CANBERRA CIVIC DOORDASH DUMMY
6328,Drinks,Sports drinks,Sports drink,Tropical Fruit Sports Drink,2.25,$3.75 per 1L,600mL,0,1,False,3.80,Gatorade,5468917P,3.75,1L,CANBERRA CIVIC DOORDASH DUMMY


In [40]:
df.shape

(6330, 16)

In [42]:
df.head(2)

,Category,Sub_category,Product_Group,Product_Name,Package_price,Price_per_unit,package_size,is_estimated,is_special,in_stock,Retail_price,Brand,Sku,unit_price,unit_price_unit,city
0,Meat & seafood,Poultry,Drumsticks & maryland,RSPCA Approved Chicken Drumsticks,7.0,$3.50 per 1Kg,2kg,0,1,False,8.0,Coles,2263260P,3.5,1Kg,TEMPE
1,Meat & seafood,Beef & veal,Beef steaks,Beef Chuck Casserole Steak,17.1,$18.00 per 1Kg,approx. 950g,1,1,False,19.0,Coles,3331172P,18.0,1Kg,TEMPE


In [47]:
import category_encoders as ce
ordenc=ce.OrdinalEncoder(cols=["unit_price_unit","package_size","Price_per_unit","Brand","Sku","city"])
df=ordenc.fit_transform(df)

In [ ]:
df["Category"],df["Sub_category"],df["Product_Group"],df["Product_Name"],df["in_stock"]

In [52]:
df = df.apply(label_encoder.fit_transform)

In [53]:
df

,Category,Sub_category,Product_Group,Product_Name,Package_price,Price_per_unit,package_size,is_estimated,is_special,in_stock,Retail_price,Brand,Sku,unit_price,unit_price_unit,city
0,0,0,0,0,103,0,0,0,1,0,168,0,0,207,0,0
1,0,1,1,1,161,1,1,1,1,0,255,0,1,318,0,0
2,0,1,2,1,161,1,1,1,1,0,255,0,1,318,0,0
3,0,2,3,2,126,2,2,1,1,0,202,0,2,357,0,0
4,0,3,4,3,139,3,3,1,1,0,225,0,3,312,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6325,4,22,59,184,155,130,84,0,1,0,259,104,186,281,1,73
6326,4,22,59,124,115,97,36,0,1,0,220,85,125,221,1,73
6327,4,21,58,409,27,162,8,0,1,0,69,131,415,215,1,73
6328,4,21,58,567,27,162,8,0,1,0,69,131,576,215,1,73
